<a href="https://colab.research.google.com/github/SezerBugday/photogrammetry_meshroom_colab_flask/blob/main/Bitirme.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python3 main.py

In [2]:
from pyngrok import ngrok

import os  
import shutil  


from flask import Flask, flash, request, redirect, url_for,render_template
from werkzeug.utils import secure_filename

port_no = 5000

import sqlite3 as sql


con = sql.connect("/content/photogrammetry_meshroom_colab_flask/sezerdb.db", check_same_thread=False)
cursor = con.cursor()

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import sys

import random


In [10]:
from flask import Flask, flash, request, redirect, url_for, render_template
from werkzeug.utils import secure_filename

TEMPLATE = "/content/photogrammetry_meshroom_colab_flask/templates"
app = Flask(__name__, template_folder=TEMPLATE)
app._static_folder = '/content/static/'

#bootsrapt = Bootstrap(app)
islem_durumu=False;


app.config['UPLOAD_FOLDER'] = "/content/kirli"

ngrok.set_auth_token("2DJyoS7QMsne6ZsjPBWlik5uBBv_37hajTNHz6NFWXeczXf7A")
public_url = ngrok.connect(port_no).public_url

print(f"To acces the Gloable link please click {public_url}")


@app.route("/")
def main_page():

    print("Anasayfaya donuldu")
    return render_template("HomePage.html")

@app.route("/uploadimages", methods=["GET", "POST"])
def upload_file():
    print("resimler secildi")
    if request.method == 'POST':
        files = request.files.getlist("image")
        for file in files:
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], file.filename))
            print("Görseller kaydedildi")
            return redirect(url_for('Show_Process'))
    return render_template("UploadImages.html")
  
@app.route("/process")
def Show_Process():
  if(islem_durumu==False):
    return redirect(url_for('Process'))
  return render_template("processing.html")
  print("isleniyor")


@app.route("/sezer")
def Process():
  return render_template("showModel.html")
  return "çalisti"
  os.system("rembg p /content/kirli   /content/temiz")
  os.system("./meshroom/Meshroom-2019.1.0/meshroom_photogrammetry --input /content/temiz --output /content/modelFolder")
  os.system("obj2gltf -i /content/modelFolder/model.obj -o /content/static/kup.glb ")
  

  
@app.route("/SingUp", methods=["GET", "POST"])
def SingUp():
   if request.method == 'POST':
        name = request.form.get("name")
        surname = request.form.get("surname")
        global email
        email = request.form.get("email")
        password = request.form.get("password")
        print("before db")
        print(name)
        try:
          cursor.execute("insert into Users values(?,?,?,?,?,?)",(name, surname, password,email,0,None))
          con.commit()
          print("Yeni Kullanıcı kaydedildi")
        except:
            print("Kaydedilmedi")
        try:
          global user_id
          cursor.execute("SELECT CustomerID  FROM  Users WHERE Email = ? and Password = ? ", (email, password))
          veri = cursor.fetchall()
          user_id= veri[0][0]
          print(veri[0][0])
          print("Kullanıcı Id alındı")
        except:
            print("Id alınamadi")   
        # Gmail email sunucusuna bağlanıyoruz
        try:
            global ver_code
            ver_code = random.randint(100000,999999)
            print("vercode", ver_code)
            mail = smtplib.SMTP("smtp.gmail.com",587)
            mail.ehlo()
            mail.starttls()
            mail.login("sezerbugday@gmail.com", "vqrwxdipzjoftucq")
            mesaj = MIMEMultipart()
            mesaj["From"] = "3D AR Object Platform "           # Gönderen
            mesaj["Subject"] = "Verification Code"   # Konusu
            mesaj["To"] = email
            #body = "Your Verification Code :  "+ str(ver_code)
            body = "To Verify email please click "+str(public_url)+"/Deneme/"+str(user_id) 
            body_text = MIMEText(body, "plain")  #
            mesaj.attach(body_text)
            mail.sendmail(mesaj["From"], mesaj["To"], mesaj.as_string())
            print("Mail başarılı bir şekilde gönderildi.")
            mail.close()
            return render_template("MailVerifed.html", mail_adress = mesaj["To"])
            #return render_template("SentVerificationCode.html")
        # Eğer mesaj gönderirken hata olursa, hata mesajını konsole yazdırıyorum.
        except:
            print("Hata:", sys.exc_info()[0])
            
   return render_template("SingUp.html")

@app.route("/SentLink/<string:numara>")
def SentLinkF(numara):
         # Gmail email sunucusuna bağlanıyoruz
        try:
            mail = smtplib.SMTP("smtp.gmail.com",587)
            mail.ehlo()
            mail.starttls()
            mail.login("sezerbugday@gmail.com", "vqrwxdipzjoftucq")
            mesaj = MIMEMultipart()
            mesaj["From"] = "3D AR Object Platform "           # Gönderen
            mesaj["Subject"] = "Verification Link"   # Konusu
            mesaj["To"] = "sezerbugday@gmail.com"
            body = "To acces the Gloable link please click "+str(public_url)+str(numara)
 
            body_text = MIMEText(body, "plain")  #
            mesaj.attach(body_text)
            mail.sendmail(mesaj["From"], mesaj["To"], mesaj.as_string())
            print("Mail başarılı bir şekilde gönderildi.")
            mail.close()
            return "mail_gönderildi"
            #return render_template("MailVerifed.html", mail_adress = mesaj["To"])
        # Eğer mesaj gönderirken hata olursa, hata mesajını konsole yazdırıyorum.
        except:
            print("Hata:", sys.exc_info()[0])
            
        return render_template("SingUp.html")

@app.route("/SentCode", methods=["GET", "POST"])
def SentCode():
  if request.method == 'POST':
    code = request.form.get("sifre")
    print("gelen kod", ver_code)
    print("girilen kod", code)

    if(int(code) == ver_code):
      cursor.execute("UPDATE Users SET Is_Active = 1 WHERE Email = email" )
      con.commit()
      return render_template("Login.html",yazi="Tebrikler mailiniz onaylandı şimdi giriş yapabilirsiniz.")
    else:
      return render_template("SentVerificationCode.html",yazi="Mail onaylanmadı kodu tekrar giriniz.")
  return render_template("SentVerificationCode.html")


@app.route("/Deneme/<string:kim_bu>")
def Deniyorum(kim_bu):
    kim_bu= user_id
    try:
      cursor.execute("UPDATE Users SET Is_Active = 1 WHERE CustomerID = ?", (kim_bu,))
      con.commit()
      return render_template("Login.html",yazi="Kullanıcı maili onaylandı. Giriş Yapabilirsiniz.")
    except:
      print("Sorgu başarısız")
   


@app.route("/CheckLogin", methods=["GET", "POST"])
def CheckLogin():
  if request.method == 'POST':
    user_mail = request.form.get("email")
    user_password = request.form.get("password")
    print("Check",user_mail)
    print("Check2",user_password)
    cursor.execute("SELECT * FROM  Users WHERE Email = ? and Password = ? ", (user_mail, user_password))
    veri = cursor.fetchall() 
    if len(veri) != 0:
      return render_template("AddProduct.html")
    else:
      return render_template("Login.html",yazi="Kullanıcı maili veya şifre yanlış tekrar deneyiniz")
  
@app.route("/SaveProduct", methods=["GET", "POST"])
def SaveProduct():
  if request.method == 'POST':
    product_name = request.form.get("product_name")
    product_info = request.form.get("product_info")
    user_place1 = request.form.get("place1")
    user_place2= request.form.get("place2")
    user_place3 = request.form.get("place3")
    global product_id
    product_id = random.randint(100000,999999)

    print(product_id)
    ozel_link =str(public_url)+"/Products/"+str(product_id)
    cursor.execute("insert into Products values(?,?,?,?,?,?,?,?)",(product_name,user_id,product_id, product_info, ozel_link,user_place1,user_place2,user_place3))
    con.commit()
    
    return render_template("ProductPage.html",str(product_id)+".glb",isim=product_name,bilgi=product_info,ozel_url=ozel_link, url1=user_place1,url2=user_place2,url3=user_place3)
   
@app.route("/Products/<string:numara>") #
def Products(numara):
    numara= product_id
    print("sorgu öncesi")
    cursor.execute("SELECT * FROM  Products WHERE ProductID = ? ", (numara,))
    veri = cursor.fetchall() 
    print("sorgu sonrası")
    if len(veri) != 0:
      print("veri bulundu")

      return render_template("ProductPage.html", isim=veri[0][0],bilgi=veri[0][3], 
        ozel_url=veri[0][4], url1=veri[0][5],
        url2=veri[0][6],url3=veri[0][7])
    else:
      return "UrunBulunmadi"
       
if __name__ == "__main__":
    app.run()

To acces the Gloable link please click http://6a63-104-196-110-222.ngrok.io
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
INFO:werkzeug:127.0.0.1 - - [14/Jan/2023 11:10:25] "GET / HTTP/1.1" 200 -


Anasayfaya donuldu


INFO:werkzeug:127.0.0.1 - - [14/Jan/2023 11:10:25] "GET /static/logo.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Jan/2023 11:10:25] "GET /static/Vr_image.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Jan/2023 11:10:26] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [14/Jan/2023 11:10:26] "GET /uploadimages HTTP/1.1" 200 -


resimler secildi


...

INFO:werkzeug:127.0.0.1 - - [14/Jan/2023 11:10:32] "GET /process HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [14/Jan/2023 11:10:32] "GET /sezer HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Jan/2023 11:10:34] "GET /static/kup.glb HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Jan/2023 11:10:38] "GET /SingUp HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Jan/2023 11:10:38] "GET /sgn-up.css HTTP/1.1" 404 -


before db
sezer
Yeni Kullanıcı kaydedildi
63
Kullanıcı Id alındı
vercode 463593


INFO:werkzeug:127.0.0.1 - - [14/Jan/2023 11:11:10] "POST /SingUp HTTP/1.1" 200 -


Mail başarılı bir şekilde gönderildi.


INFO:werkzeug:127.0.0.1 - - [14/Jan/2023 11:11:39] "GET /Deneme/63 HTTP/1.1" 200 -


In [4]:


# Gmail email sunucusuna bağlanıyoruz
try:
    mail = smtplib.SMTP("smtp.gmail.com",587)
    mail.ehlo()
    mail.starttls()
    mail.login("sezerbugday@gmail.com", "vqrwxdipzjoftucq")

    mesaj = MIMEMultipart()
    mesaj["From"] = "3D AR Object Platform "           # Gönderen
    mesaj["Subject"] = "Verification Code"    # Konusu
    mesaj["To"] = "sezerbugday2@gmail.com "
    body = " Bu mesaj sezer tarafından python ile gönderilmiştir "

    body_text = MIMEText(body, "plain")  #
    mesaj.attach(body_text)

    mail.sendmail(mesaj["From"], mesaj["To"], mesaj.as_string())
    print("Mail başarılı bir şekilde gönderildi.")
    mail.close()

# Eğer mesaj gönderirken hata olursa, hata mesajını konsole yazdırıyorum.
except:
    print("Hata:", sys.exc_info()[0])

Mail başarılı bir şekilde gönderildi.


In [ ]:
numara= user_id
print(numara)
print(user_id)
print("sorgu öncesi")
cursor.execute("SELECT * FROM  Products WHERE ProductID = 61 ")
veri = cursor.fetchall() 
print(veri)
cursor.execute("Select * From Users")
user_veri = cursor.fetchall() 
print(user_veri)

61
61
sorgu öncesi
[]
[('sezer', 'bugday', '123456', 'sezerbugday2@gmail.com', 1, 55), ('deneme2', 'deneme2', 'sezer', 'sezerbugday2@gmail.com', 1, 56), ('deneme3', 'deneme3', '0000', 'sezerbugday2@gmail.com', 1, 57), ('sezer', 'gdfgdf', '111', 'sezerbugday2@gmail.com', 1, 58), ('nisa', 'dalkiran', '2001', 'sezerbugday2@gmail.com', 1, 59), ('sezer', 'sezer', '000000', 'sezerbugday2@gmail.com', 1, 60), ('sd', 'dsds', '999999', 'sezerbugday2@gmail.com', 1, 61)]


In [ ]:
password="123456"
cursor.execute("SELECT CustomerID  FROM  Users WHERE Email = ? and Password = ? ", (email, password))
veri = cursor.fetchall()
print(veri[0][0])

55


In [ ]:
cursor.execute("insert into Users values(?,?,?,?,?,?)",("name", "surname", "123456","sezerbugday2@gmail.com",0,None))
con.commit()

In [ ]:
cursor.execute("SELECT * FROM  Users WHERE Email = 'sezerbugday2@gmail.com' and Password = 123456")
veri = cursor.fetchall() 
print(veri)
print(len(veri))

[('name', 'surname', '123456', 'sezerbugday2@gmail.com', 1, 35), ('dsfd', 'sdfdsf', '123456', 'sezerbugday2@gmail.com', 1, 40), ('fsddfs', 'sdfsdfds', '123456', 'sezerbugday2@gmail.com', 1, 43)]
3


In [ ]:
def VeriSil():
    cursor.execute("DELETE FROM Products")
    con.commit()
VeriSil()

In [ ]:
cursor.execute("SELECT Products.CustomerID,Users.name,Users.Surname,Products.Name, Products.ProductID FROM Users INNER JOIN Products ON Users.CustomerID = Products.CustomerID ;")
veri = cursor.fetchall() 
print(veri)


[(55, 'sezer', 'bugday', 'urun', 6)]


In [ ]:
cursor.execute("SELECT * From Products")
veri = cursor.fetchall() 
print(veri)

[]


In [ ]:
password="10"
cursor.execute("SELECT *  FROM  Products WHERE  ProductID = ?  ", (password,))
veri = cursor.fetchall()
print(veri[0][3])

telefon bilgisi


In [ ]:
%cd /content/photogrammetry_meshroom_colab_flask

/content/photogrammetry_meshroom_colab_flask


In [ ]:
!git status


On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   sezerdb.db
	modified:   templates/AddProduct.html
	modified:   templates/Profil.html

no changes added to commit (use "git add" and/or "git commit -a")


In [ ]:
!git add --all  

In [ ]:
!git config --global user.email "sezerbugday2@gmail.com"

In [ ]:
!git commit -a -m "urun tablosu eklendi"


[main 8f0acd0] urun tablosu eklendi
 3 files changed, 24 insertions(+), 9 deletions(-)


In [ ]:
!git branch

* main


In [ ]:
!git push origin main

Counting objects: 6, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (6/6), done.
Writing objects: 100% (6/6), 1.24 KiB | 1.24 MiB/s, done.
Total 6 (delta 5), reused 0 (delta 0)
remote: Resolving deltas: 100% (5/5), completed with 5 local objects.
To https://github.com/SezerBugday/photogrammetry_meshroom_colab_flask.git
   5c6afc6..8f0acd0  main -> main
